In [3]:
import pandas as pd
import numpy as np
import os
from datetime import *
from fn import *
from o_rpa_def import *
from o_rpa_defonly import *
import o_time as oT
from collections import defaultdict

def inner_list_to_dic(dic):
    for key, value in dic.items(): 
        dic[key] = set(value)
    return dic

def joinls(l1,l2):
    if len(l1)!=0 and len(l2)!=0:
        lss = []
        for i in l1:
            for j in l2:
                lss.append(str(i) + '$' + str(j))
        return lss
    elif len(l1)!=0 and len(l2)==0:
        return l1
    elif len(l1)==0 and len(l2)!=0:
        return l2
    else:
        return l1
    
def colchk(df):
    mcols = ['EQUIPMENTKEY','SITECODE','SUMMARY','ALERTKEY','LASTOCCURRENCE','CLEARTIMESTAMP']
    ocols = ['RESOURCE','CUSTOMATTR15','SUMMARY','ALERTKEY','LASTOCCURRENCE','CLEARTIMESTAMP']
    df = df.rename (columns=str.upper)
    cols = df.columns.to_list()
    if cols.count('SITECODE') != 0:
        df = df.rename(columns={'SITECODE':'CUSTOMATTR15'})
    if cols.count('EQUIPMENTKEY') != 0:
        df = df.rename(columns={'EQUIPMENTKEY':'RESOURCE'})
    for i in ocols:
        if ocols.count(i) == 0:
            print('must have column needs in table: but missing !',chr(10),mcols,chr(10),'exiting .....')
            exit(0)
    else:
        sx = chrstream()
        omnm(sx)
        print(chr(10))
        return df

def nested_dic_add(dc, k, v):
    #map based on child dictionary and add new key value/append by key matching on clild
    if len(dc)>0:
        ln = len(list(dc))
        for i in dc:
            if k in list(dc[i]):
                dc[i][k] = v if not list(dc[i]) else dc[i].get(k, []) + [v]
                return dc
        else:
            if isinstance(v, list):
                dc[ln+1] = {k:v}
            else:
                v1 = [v]
                dc[ln+1] = {k:v1}
            return dc
    else:
        if isinstance(v, list):
            dc[1] = {k:v}
        else:
            v1 = [v]
            dc[1] = {k:v1}
        return dc    

def mprep(df, dic, oncol):
    heap = ""
    gdc = defaultdict(dict)
    for k, v in dic.items():
        dff = df[df[k].isin(v)]
        dff = dff.reset_index()
        for i in range(len(v)):
            hp = str(k) + "|" + str(dff.shape[0]) + chr(10) + dff[oncol].str.contains(v[i]).cat(sep=chr(10))
            if heap == "":
                heap = hp
            else:
                heap = heap + chr(10) + chr(10) + hp
        else:
            print(heap)
    
class o_rpa:
    def __init__(self, mdf):
        self.odb = omdb()
        self.df0 = colchk(mdf)
        self.df1 = catmap(self.df0, self.odb)
        self.df = self.df1
        self.pickcols = "CDLO"
        self.msgthread = defaultdict(dict)
        self.mth = []
        self.cond = defaultdict(dict)
        self.lsky = []
        self.lsvl = []
        self.lsx = []
        print(self.df.columns)
    def regionwise(self, Name=[]):
        self.msgthread = list(zn_dic())
    def techwise(self):
        self.msgthread = {'CAT': {'2G':"","3G":"","4G":""}}
    def msgcol(self, lscols):
        fault = 0
        col = self.df.columns.to_list()
        for i in lscols:
            if col.count(i)==0:
                fault = 1
        else:
            if fault == 0:
                self.df = self.df.assign(pk = self.df[lscols].apply(lambda x: '- '.join(x.values.astype(str)), axis=1))
                self.pickcols = "pk"
            else:
                print("!!!!!!!!!!!!!!!!! column name not found",chr(10),self.df.columns)
    def csvmap(self, csvpath, match_col_name, column_to_pick = []):
        try:
            try:
                dff = pd.read_csv(csvpath)
            except:
                dff = csvpath
            if column_to_pick is not None:
                dff = dff[column_to_pick]
            ndf = self.df
            self.df = ndf.merge (dff, on=match_col_name)
        except:
            print('csvpath not found')
    def pnt(self):
        print(self.df)
    def getdf(self, current=True):
        if current==False:
            return self.df0
        else:
            return self.df
    def sample(self):
        print(self.df.head(5))
    def summary(self):
        print("COLUMN:", self.df.columns, chr(10))
        print("Current Row: ", self.df.shape[0],'--',"Row Main:", self.df1.shape[0], chr(10))
        print("filtering conditions : ", dict((k, v) for k, v in self.cond.items()))
        print("msgformat by pickcols: ", 'CUSTOMATTR15','LASTOCCURRENCE')
        print("msgthread: ", self.msgthread)
    def condition(self,colname,colval):
        ndc = nested_dic_add(self.cond,colname,colval)
        self.cond = ndc.copy()
    def rwise(self, whichzn=False):
        if whichzn == False:
            whichzn = "ALL"
        if len(self.cond) == 1:
            colsMain = list(self.cond[1])
            rval = parsing(self.df, whichzn,[],colsMain[0], self.cond[1].get(colsMain[0]), False, False)
        elif len(self.cond) == 2:
            colsMain = list(self.cond[1])
            cl = list(self.cond[2])
            rval = parsing(self.df, whichzn,self.pickcols,colsMain[0], self.cond[1].get(colsMain[0]), cl[0], 
                           self.cond[2].get(cl[0]))
        else:
            print('under development')
    def regionwise_count(self, list_cat):
        rv = zonewise_count(self.df1, list_cat)
        print('----------------------------------------')
        return rv
    def update_cond_1(self, ky, vl):
        self.cond.setdefault(ky, []).append(vl)
    def update_cond_2(self, ky, vl):
        if isinstance(vl,list):
            self.cond[ky] = vl if not list(self.cond) else self.cond.get(ky, []) + vl
        else:
            vlx = [vl]
            self.cond[ky] = vlx if not list(self.cond) else self.cond.get(ky, []) + vlx
    def timecal(self, start_time_colname, end_time_colname=False):
        self.df = oT.dfdiff(self.df, start_time_colname, end_time_colname)
        print("new column name is 'DUR'")
    def timefmt(self, colname, fmt="%Y/%m/%d %H:%M:%S"):
        if len(fmt) <= 9:
            self.df[colname] = self.df.apply(lambda x: oT.sec_to_dur(x[colname]), axis = 1)
            print(self.df[colname])
        else:
            self.df[colname] = oT.datetime_convert_format(self.df, colname, fmt)
            print(self.df[colname])
    def add2(self, k, v):
        self.df = self.df[self.df[k].isin(v)]
        self.lsky.append(k)
        self.lsvl.append(v)
        xk = joinls(self.lsx,v)
        self.lsx = xk
        print(self.lsx)
        try:
            self.df = self.df[self.df[k].isin(v)]
        except:
            print('except trigger')
    def gen3(self):
        hpf = chr(10)
        ndf = self.df
        print(ndf.columns)
        for i in range(len(self.lsx)):
            spl = self.lsx[i].split('$')
            if len(self.lsky) == len(spl):
                xdf = self.df
                for n in range(len(spl)):
                    k = self.lsky
                    v = spl[n]
                    xdf = xdf[xdf[k[n]].isin([v])]
                else:
                    x11 = " | ".join(spl) + ": " + str(xdf.shape[0])
                    if xdf.shape[0] != 0:
                        x11 = x11 + chr(10) + xdf[self.pickcols].str.cat(sep=chr(10))
                    print(x11, chr(10))
    def pntcond(self):
        L = list(self.cond.values())
        Lx = []
        if (len(L[0]))<=1:
            L1 = [item for sublist in L[0] for item in sublist]
            Lx.append(L1)
        else:
            Lx.append(L1)

df = pd.read_csv(os.getcwd() + "\\sclick.csv")  # data source, 
dfp1p2 = pd.read_csv (os.getcwd() + "\\site_p1p2.csv")
xx = o_rpa(df)
xx.timecal('LASTOCCURRENCE')
xx.timefmt('DUR','%H:%M:%S')
xx.update_cond_1('sZone',['COM','NOA'])
xx.update_cond_1('CAT',['DL'])
#xx.update_cond_1('P1P2',['','P2'])
#xx.sample()
#xx.msgcol(['CUSTOMATTR15','LASTOCCURRENCE','P1P2'])
xx.pntcond()

xx.gen3()
#rval = xx.rwise()
#ST = xx.regionwise_count(['2G','3G','4G','MF','DL'])
#print(ST)
#dc = {'sZone':{0:{'COM':{'CAT':{0:'2G'}}}}}
#print(dc)
#slicedf(dx,dc)



- - - - - - - - - - - - - - - - - -
[      ____  __  __   _          
      / __ \|  \/  | | |         
     | |  | | \  / | | |         
     | |  | | |\/| | | |         
     | |__| | |  | | | |         
      \____/\_|  |_| |_|        ]

- - - - - - - - - - - - - - - - - -


Index(['RESOURCE', 'CUSTOMATTR15', 'SUMMARY', 'ALERTKEY', 'LASTOCCURRENCE',
       'CLEARTIMESTAMP', 'CAT', 'CODE', 'sCode', 'sZone', 'CODECAT', 'ZNCAT',
       'CDLO', 'P1P2', 'ATRB', 'Name'],
      dtype='object')
new column name is 'DUR'
0       0:22:26
1        0:6:36
2        0:1:36
3        0:7:31
4        0:2:39
         ...   
2438     0:16:3
2439    0:16:11
2440     0:16:4
2441     0:16:2
2442     0:16:2
Name: DUR, Length: 2443, dtype: object
Index(['RESOURCE', 'CUSTOMATTR15', 'SUMMARY', 'ALERTKEY', 'LASTOCCURRENCE',
       'CLEARTIMESTAMP', 'CAT', 'CODE', 'sCode', 'sZone', 'CODECAT', 'ZNCAT',
       'CDLO', 'P1P2', 'ATRB', 'Name', 'DUR'],
      dtype='object')
